# Task B1
We choose referencing `child`document model for taskB. 

##Model Structue 
###Climate Collection
{'_id': ObjectId('5aefae877c376e600548a8b7'),
 'Station': 948702,
 'Date': '2017-12-15',
 'Air Temperature(Celcius)': 18,
 'Relative Humidity': 52.0,
 'WindSpeed  (knots)': 7.1,
 'Max Wind Speed': 14.0,
 'MAX': '74.5*',
 'MIN': '53.1',
 'Precipitation': '0.00I',
 'fires': [ObjectId('5aefae877c376e6005489d05'),
  ObjectId('5aefae877c376e6005489d06'),
  ObjectId('5aefae877c376e6005489d07'),
  ObjectId('5aefae877c376e6005489d08')]}

###Fire Collection:
{'_id': ObjectId('5aefae877c376e6005489cef'),
 'Latitude': -37.966,
 'Longitude': 145.051,
 'Surface Temperature (kelvin)': 341.8,
 'Datetime': '2017-12-27T04:16:51',
 'Power': 26.7,
 'Confidence': 78,
 'Date': '2017-12-27',
 'Surface Temperature (Celcius)': 68}
 
## Justification

### There are five main reasons why we chose this model: 

#### 1. One day has one climate record but may has 0 or up to around 360 fire records sometimes. 360 records for a single day are not so large, but if we use the embedding document (embedding the fire document inside the climate document), then some of the climate document may require a big array to carry all details of the 400 fire records. In contrast, child referencing document model can solve this issue, because only the object id of related fire data would be included inside the array of the climate document. If we choose the other way around, using parent reference, that is keeping the _id of a climate record in all related fire record file, then there will be a lot of duplication, and since our dataset is not that big, we abandoned this design.

    The number of fires for some days are shown below.

{'Date': '2017-04-13', 'NumberOfFire': 357}
{'Date': '2017-04-18', 'NumberOfFire': 325}
{'Date': '2017-05-04', 'NumberOfFire': 135}
{'Date': '2017-04-06', 'NumberOfFire': 118}
{'Date': '2017-05-10', 'NumberOfFire': 114}
{'Date': '2017-05-15', 'NumberOfFire': 102}
{'Date': '2017-04-04', 'NumberOfFire': 89}
{'Date': '2017-04-03', 'NumberOfFire': 72}
{'Date': '2017-04-15', 'NumberOfFire': 69}
{'Date': '2017-04-12', 'NumberOfFire': 69}

#### 2. When performing the join operator, it is much faster to join by the fire’s object id than by date field since the index for the object id field is automatically done by mongodb.Also, it is clearer to keep track of the relation between 2 collections.

#### 3. Another advantage is it is easy to perform some queries such as summing up the number of fires per day using such a model 

#### 4. The Requirement for updating data is seldom since all the fire data are collected by the sensor automatically and the error rate is quite low. In addition, weather data is based on forecast. That is to say, it does not require for updating.

#### 5. Two separated collections are needed, as data analysts sometimes would perform queries for climate data and fire data independently in order to find out the valuable information. Also the frequency of performing queries for fire collection alone is quite high.


# Task B2 & B3
Write a python program that will read the data from Fire data-Part1 and Climate data - Part1 and load them to the new database created in Task B2.

In [1]:
from pprint import pprint
import csv
firePath = './FireData-Part1.csv'
climatePath = './ClimateData-Part1.csv'

In [2]:
def read_to_list(path):
    with open(path,'r') as f:
        reader = csv.reader(f)
        return list(reader)

In [3]:
climateData = read_to_list(climatePath)[1:]
fireData = read_to_list(firePath)[1:]

In [4]:
#make connection to mongod
from bson.objectid import ObjectId
import random, string
import random
import pymongo
from pymongo import MongoClient
import multiprocessing as mp
from pymongo.cursor import Cursor
client = MongoClient()
#drop the database if it already existed
client.drop_database('fit5148_db')
#get the database
db = client.fit5148_db
#create two new collections 
Fire = db.fire
Climate = db.climate

In [5]:
#create fires collection and insert fire documents into fires collection 
for fire in fireData:
    Fire.insert_one(
        {
         "_id" : ObjectId(''.join(random.choices(string.ascii_letters + string.digits, k=12)).encode()), 
         "Latitude": float(fire[0].strip()),
         "Longitude": float(fire[1].strip()),
         "Surface Temperature (kelvin)": float(fire[2].strip()),  
         "Datetime": fire[3].strip(),  
         "Power": float(fire[4].strip()),   
         "Confidence": int(fire[5].strip()),  
         "Date": fire[6].strip(),  
         "Surface Temperature (Celcius)":  int(fire[7].strip()),  
        })


In [6]:
#create climates collection and insert climate documents into climates collection
for climate in climateData:
    Climate.insert_one(
        {
         "Station": int(climate[0].strip()),
         "Date": climate[1].strip(),
         "Air Temperature(Celcius)": int(climate[2].strip()),  
         "Relative Humidity": float(climate[3].strip()),  
         "WindSpeed  (knots)": float(climate[4].strip()),   
         "Max Wind Speed": float(climate[5].strip()),  
         "MAX": climate[6].strip(),  
         "MIN": climate[7].strip(),  
         "Precipitation": climate[8].strip()
        })

In [7]:
#find the array of all associated object ids for referencing the child 
#and update the array for each climate document
#get all fire documents from fire collection
allFires = Fire.find({})
#get all climate documents from climate collection
allClimates = Climate.find({})
fireIds = []
for climate in allClimates:       
    results = Fire.find({"Date": climate['Date']})
    for doc in results:
        fireIds.append(doc['_id'])
    Climate.update_one({"_id": climate.get('_id')}, {"$set": {"fires": fireIds}}) 
    fireIds = []

In [8]:
#inspect data model of Climate
Climate.find_one({"Date":"2017-12-15"})

{'_id': ObjectId('5b0167e57c376e3639e0039d'),
 'Station': 948702,
 'Date': '2017-12-15',
 'Air Temperature(Celcius)': 18,
 'Relative Humidity': 52.0,
 'WindSpeed  (knots)': 7.1,
 'Max Wind Speed': 14.0,
 'MAX': '74.5*',
 'MIN': '53.1',
 'Precipitation': '0.00I',
 'fires': [ObjectId('693839514c52683454324a4b'),
  ObjectId('354876464441693577726773'),
  ObjectId('76453165416c61484c5a514c'),
  ObjectId('50436f3841437a546b774e52')]}

In [9]:
#inspect data model of Fire
Fire.find_one()

{'_id': ObjectId('76774e56656f6b7457433443'),
 'Latitude': -37.966,
 'Longitude': 145.051,
 'Surface Temperature (kelvin)': 341.8,
 'Datetime': '2017-12-27T04:16:51',
 'Power': 26.7,
 'Confidence': 78,
 'Date': '2017-12-27',
 'Surface Temperature (Celcius)': 68}

# Task B4
Write queries to answer the Tasks A2-A8 on the new database. You need to write the queries as a python program using pymongo library in Jupyter Notebook. Please choose ONE of the Task A2-A8 and implement it in parallel.

### A2 Find climate data on 15th December 2017

In [10]:
from pprint import pprint
results = Climate.find({"Date":"2017-12-15"})
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'MAX': '74.5*',
 'MIN': '53.1',
 'Max Wind Speed': 14.0,
 'Precipitation': '0.00I',
 'Relative Humidity': 52.0,
 'Station': 948702,
 'WindSpeed  (knots)': 7.1,
 '_id': ObjectId('5b0167e57c376e3639e0039d'),
 'fires': [ObjectId('693839514c52683454324a4b'),
           ObjectId('354876464441693577726773'),
           ObjectId('76453165416c61484c5a514c'),
           ObjectId('50436f3841437a546b774e52')]}


### A3 Find the latitude, longitude and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [11]:
results = Fire.find(
    {"Surface Temperature (Celcius)":{"$gt":65,"$lt":100}},
    {"Longitude":1,"Latitude":1,"Confidence":1,"_id":0}
)
for document in results: 
    pprint(document)

{'Confidence': 78, 'Latitude': -37.966, 'Longitude': 145.051}
{'Confidence': 86, 'Latitude': -35.543, 'Longitude': 143.316}
{'Confidence': 93, 'Latitude': -37.875, 'Longitude': 142.51}
{'Confidence': 95, 'Latitude': -37.613, 'Longitude': 149.305}
{'Confidence': 90, 'Latitude': -37.624, 'Longitude': 149.314}
{'Confidence': 93, 'Latitude': -38.057, 'Longitude': 144.211}
{'Confidence': 92, 'Latitude': -37.95, 'Longitude': 142.366}
{'Confidence': 100, 'Latitude': -36.282, 'Longitude': 146.157}
{'Confidence': 100, 'Latitude': -37.634, 'Longitude': 149.237}
{'Confidence': 98, 'Latitude': -37.605, 'Longitude': 149.302}
{'Confidence': 99, 'Latitude': -37.6, 'Longitude': 149.325}
{'Confidence': 95, 'Latitude': -37.618, 'Longitude': 149.281}
{'Confidence': 100, 'Latitude': -37.606, 'Longitude': 149.312}
{'Confidence': 97, 'Latitude': -37.61, 'Longitude': 149.307}
{'Confidence': 99, 'Latitude': -37.609, 'Longitude': 149.32}
{'Confidence': 93, 'Latitude': -37.945, 'Longitude': 144.354}
{'Confidenc

### A4 Find surface temperature (°C), air temperature (°C), relative humidity and maximum wind speed on 15th and 16th of December 2017

In [12]:
results = Climate.aggregate([
{
    "$unwind":"$fires"
},
{"$lookup":{
    "from":"fire",
    "localField":"fires",
    "foreignField":"_id",
    "as":"Fire"}
},
{
 "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$Fire", 0 ] }, "$$ROOT" ] } }
},

{
    "$project": {"Date":1,"Surface Temperature (Celcius)": 1,
               "Air Temperature(Celcius)":1,"Relative Humidity":1,
               "Max Wind Speed":1,"_id":0} 
},
{
    "$match":{"$or":[{"Date":"2017-12-15"},{"Date":"2017-12-16"}]}
}
])
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Max Wind Speed': 14.0,
 'Relative Humidity': 52.0,
 'Surface Temperature (Celcius)': 42}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Max Wind Speed': 14.0,
 'Relative Humidity': 52.0,
 'Surface Temperature (Celcius)': 36}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Max Wind Speed': 14.0,
 'Relative Humidity': 52.0,
 'Surface Temperature (Celcius)': 38}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Max Wind Speed': 14.0,
 'Relative Humidity': 52.0,
 'Surface Temperature (Celcius)': 40}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-16',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 43}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-16',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface Temperature (Celcius)': 33}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-16',
 'Max Wind Speed': 13.0,
 'Relative Humidity': 53.7,
 'Surface

### A5 Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [13]:
results = Climate.aggregate([
{
    "$unwind":"$fires"
},
{"$lookup":{
    "from":"fire",
    "localField":"fires",
    "foreignField":"_id",
    "as":"Fire"}
},
{
 "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$Fire", 0 ] }, "$$ROOT" ] } }
},
{ 
    "$project": {"Datetime":1,"Air Temperature(Celcius)":1,
    "Surface Temperature (Celcius)": 1,"Confidence":1, "_id":0}
 },
{
    "$match":{"Confidence":{"$gt":80,"$lt":100}}}

])
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 20,
 'Confidence': 87,
 'Datetime': '2017-03-06T05:06:30',
 'Surface Temperature (Celcius)': 62}
{'Air Temperature(Celcius)': 20,
 'Confidence': 85,
 'Datetime': '2017-03-06T05:06:20',
 'Surface Temperature (Celcius)': 59}
{'Air Temperature(Celcius)': 19,
 'Confidence': 88,
 'Datetime': '2017-03-07T04:16:10',
 'Surface Temperature (Celcius)': 64}
{'Air Temperature(Celcius)': 23,
 'Confidence': 86,
 'Datetime': '2017-03-09T13:23:40',
 'Surface Temperature (Celcius)': 41}
{'Air Temperature(Celcius)': 19,
 'Confidence': 94,
 'Datetime': '2017-03-10T04:45:30',
 'Surface Temperature (Celcius)': 73}
{'Air Temperature(Celcius)': 19,
 'Confidence': 97,
 'Datetime': '2017-03-10T04:45:30',
 'Surface Temperature (Celcius)': 80}
{'Air Temperature(Celcius)': 19,
 'Confidence': 81,
 'Datetime': '2017-03-10T04:43:50',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 21,
 'Confidence': 94,
 'Datetime': '2017-03-12T04:33:50',
 'Surface Temperature (Celcius

{'Air Temperature(Celcius)': 19,
 'Confidence': 81,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 61}
{'Air Temperature(Celcius)': 19,
 'Confidence': 85,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 60}
{'Air Temperature(Celcius)': 19,
 'Confidence': 90,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 67}
{'Air Temperature(Celcius)': 19,
 'Confidence': 87,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 62}
{'Air Temperature(Celcius)': 19,
 'Confidence': 94,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 86}
{'Air Temperature(Celcius)': 19,
 'Confidence': 84,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 57}
{'Air Temperature(Celcius)': 19,
 'Confidence': 87,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius)': 62}
{'Air Temperature(Celcius)': 19,
 'Confidence': 86,
 'Datetime': '2017-04-06T04:20:40',
 'Surface Temperature (Celcius

{'Air Temperature(Celcius)': 16,
 'Confidence': 81,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 16,
 'Confidence': 94,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 75}
{'Air Temperature(Celcius)': 16,
 'Confidence': 81,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 16,
 'Confidence': 81,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 16,
 'Confidence': 90,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 67}
{'Air Temperature(Celcius)': 16,
 'Confidence': 86,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 60}
{'Air Temperature(Celcius)': 16,
 'Confidence': 86,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius)': 61}
{'Air Temperature(Celcius)': 16,
 'Confidence': 92,
 'Datetime': '2017-04-13T04:26:30',
 'Surface Temperature (Celcius

 'Surface Temperature (Celcius)': 61}
{'Air Temperature(Celcius)': 10,
 'Confidence': 91,
 'Datetime': '2017-05-04T04:45:10',
 'Surface Temperature (Celcius)': 68}
{'Air Temperature(Celcius)': 10,
 'Confidence': 91,
 'Datetime': '2017-05-04T04:45:00',
 'Surface Temperature (Celcius)': 69}
{'Air Temperature(Celcius)': 10,
 'Confidence': 81,
 'Datetime': '2017-05-04T04:44:50',
 'Surface Temperature (Celcius)': 54}
{'Air Temperature(Celcius)': 10,
 'Confidence': 81,
 'Datetime': '2017-05-04T04:44:50',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 10,
 'Confidence': 83,
 'Datetime': '2017-05-04T04:44:40',
 'Surface Temperature (Celcius)': 56}
{'Air Temperature(Celcius)': 10,
 'Confidence': 84,
 'Datetime': '2017-05-04T04:44:40',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 10,
 'Confidence': 93,
 'Datetime': '2017-05-04T04:44:40',
 'Surface Temperature (Celcius)': 72}
{'Air Temperature(Celcius)': 10,
 'Confidence': 98,
 'Datetime': '2017-05-04T04:

 'Surface Temperature (Celcius)': 60}
{'Air Temperature(Celcius)': 9,
 'Confidence': 87,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 62}
{'Air Temperature(Celcius)': 9,
 'Confidence': 84,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 58}
{'Air Temperature(Celcius)': 9,
 'Confidence': 94,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 75}
{'Air Temperature(Celcius)': 9,
 'Confidence': 82,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 56}
{'Air Temperature(Celcius)': 9,
 'Confidence': 91,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 69}
{'Air Temperature(Celcius)': 9,
 'Confidence': 91,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 69}
{'Air Temperature(Celcius)': 9,
 'Confidence': 82,
 'Datetime': '2017-05-13T04:38:40',
 'Surface Temperature (Celcius)': 55}
{'Air Temperature(Celcius)': 9,
 'Confidence': 90,
 'Datetime': '2017-05-13T04:38:40',


### A6 Find top 10 records with highest surface temperature (°C).

In [14]:
results = Climate.aggregate([
{
    "$unwind":"$fires"
},
{"$lookup":{
    "from":"fire",
    "localField":"fires",
    "foreignField":"_id",
    "as":"Fire"}
},
{
 "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$Fire", 0 ] }, "$$ROOT" ] } }
},
{ 
    "$project": { "Fire": 0,"_id":0,"fires":0}
},

{
    "$sort":{"Surface Temperature (Celcius)":-1}
},
{
    "$limit":10
}
])
for document in results: 
    pprint(document)

{'Air Temperature(Celcius)': 15,
 'Confidence': 100,
 'Date': '2017-04-18',
 'Datetime': '2017-04-18T04:52:00',
 'Latitude': -38.1665,
 'Longitude': 143.062,
 'MAX': '69.8*',
 'MIN': '46.8',
 'Max Wind Speed': 9.9,
 'Power': 239.8,
 'Precipitation': '0.00I',
 'Relative Humidity': 56.1,
 'Station': 948701,
 'Surface Temperature (Celcius)': 124,
 'Surface Temperature (kelvin)': 397.5,
 'WindSpeed  (knots)': 5.1}
{'Air Temperature(Celcius)': 16,
 'Confidence': 100,
 'Date': '2017-04-04',
 'Datetime': '2017-04-04T04:32:50',
 'Latitude': -36.343,
 'Longitude': 142.1986,
 'MAX': '73.4*',
 'MIN': '46.8',
 'Max Wind Speed': 12.0,
 'Power': 233.4,
 'Precipitation': '0.00I',
 'Relative Humidity': 47.5,
 'Station': 948701,
 'Surface Temperature (Celcius)': 123,
 'Surface Temperature (kelvin)': 396.3,
 'WindSpeed  (knots)': 5.4}
{'Air Temperature(Celcius)': 14,
 'Confidence': 100,
 'Date': '2017-05-01',
 'Datetime': '2017-05-01T04:14:20',
 'Latitude': -36.9318,
 'Longitude': 143.0907,
 'MAX': '67.

### A7 Find the number of fire in each day. You are required to only display total number of fire and the date in the output

In [15]:
results = Climate.aggregate([
{"$project":{"Date":1,"_id":0,"NumberOfFire":{"$size":"$fires"}}}
])
for document in results: 
    pprint(document)

{'Date': '2016-12-31', 'NumberOfFire': 0}
{'Date': '2017-01-02', 'NumberOfFire': 0}
{'Date': '2017-01-03', 'NumberOfFire': 0}
{'Date': '2017-01-04', 'NumberOfFire': 0}
{'Date': '2017-01-05', 'NumberOfFire': 0}
{'Date': '2017-01-06', 'NumberOfFire': 0}
{'Date': '2017-01-07', 'NumberOfFire': 0}
{'Date': '2017-01-08', 'NumberOfFire': 0}
{'Date': '2017-01-09', 'NumberOfFire': 0}
{'Date': '2017-01-10', 'NumberOfFire': 0}
{'Date': '2017-01-11', 'NumberOfFire': 0}
{'Date': '2017-01-12', 'NumberOfFire': 0}
{'Date': '2017-01-13', 'NumberOfFire': 0}
{'Date': '2017-01-14', 'NumberOfFire': 0}
{'Date': '2017-01-15', 'NumberOfFire': 0}
{'Date': '2017-01-16', 'NumberOfFire': 0}
{'Date': '2017-01-17', 'NumberOfFire': 0}
{'Date': '2017-01-18', 'NumberOfFire': 0}
{'Date': '2017-01-19', 'NumberOfFire': 0}
{'Date': '2017-01-20', 'NumberOfFire': 0}
{'Date': '2017-01-21', 'NumberOfFire': 0}
{'Date': '2017-01-22', 'NumberOfFire': 0}
{'Date': '2017-01-23', 'NumberOfFire': 0}
{'Date': '2017-01-24', 'NumberOfFi

### A8 Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output

In [16]:
results = Fire.aggregate([
{"$group":{"_id":"$Date", "AverageSurfaceTemp":{"$avg":"$Surface Temperature (Celcius)"}}},
{"$project":{"Date":"$_id","AverageSurfaceTemp":1,"_id":0}}
])
for document in results: 
    pprint(document)

{'AverageSurfaceTemp': 46.666666666666664, 'Date': '2017-03-09'}
{'AverageSurfaceTemp': 69.375, 'Date': '2017-03-10'}
{'AverageSurfaceTemp': 38.5, 'Date': '2017-03-13'}
{'AverageSurfaceTemp': 46.0, 'Date': '2017-03-15'}
{'AverageSurfaceTemp': 79.33333333333333, 'Date': '2017-03-18'}
{'AverageSurfaceTemp': 65.57142857142857, 'Date': '2017-03-19'}
{'AverageSurfaceTemp': 49.0, 'Date': '2017-03-24'}
{'AverageSurfaceTemp': 60.925925925925924, 'Date': '2017-03-28'}
{'AverageSurfaceTemp': 51.0, 'Date': '2017-03-29'}
{'AverageSurfaceTemp': 45.2, 'Date': '2017-04-02'}
{'AverageSurfaceTemp': 58.44444444444444, 'Date': '2017-04-03'}
{'AverageSurfaceTemp': 53.142857142857146, 'Date': '2017-04-05'}
{'AverageSurfaceTemp': 46.291666666666664, 'Date': '2017-04-11'}
{'AverageSurfaceTemp': 52.69565217391305, 'Date': '2017-04-12'}
{'AverageSurfaceTemp': 50.69230769230769, 'Date': '2017-04-07'}
{'AverageSurfaceTemp': 58.57983193277311, 'Date': '2017-04-13'}
{'AverageSurfaceTemp': 61.94444444444444, 'Date'

# Task B4
We choose to implement A2 in parallized way

In [17]:
cursor = Climate.find({})
climateCollection = []
for file in cursor:
    climateCollection.append(file)

In [18]:
#partition method
def rr_partition(data,n):
    """
    Perform a simple round robin partition on the given data set
    
    Parameters:
    data: the dataset to be partitioned, which is a list
    n: the number of groups that the dataset will be divided into
    
    Return:
    result: the partitioned subset of the dataset 
    """
    result = []
    for i in  range(n):
        result.append([])
    for index,element in enumerate(data):
        index_bin = index%n
        result[index_bin].append(element)
    return result
    

In [19]:
#test if the collection is sorted by date
climateCollection == sorted(climateCollection,key =lambda x: list(x.values())[2])

True

Since it is sorted by date, we can just use binary search to find the record

In [20]:
#then pick a search method:
def binary_search(data,key):
    """
    Perform binary search given certain key
    
    Parameters:
    data: the input dataset which is a list
    key: an query record
    
    Return:
    found: the mathced record and its position in a tuple, return (-1,None) if not found 
    """
    position = -1
    found = None
    upper = len(data) - 1
    lower = 0
    
    while lower <= upper and not found:
        mid = (upper + lower)//2
        if data[mid]['Date'] == key:
            found = data[mid]
            position = mid
        elif data[mid]['Date'] < key:
            lower = mid + 1
        else:
            upper = mid - 1     
    return found

In [21]:
#parallel query
def parallel_search_date(data,query,n_processor):
    """
    A method doing parallel search on a given dataset ,
    when given a search clue like a single key or a range for certain column value
    
    Parameters:
    data: the dataset to be searched, which is a list
    query: a query record
    n_processer: the number of processor to parallize the search job
    
    Return:
    results: the list of all search results in all processors
    """
    results = []
    pool = mp.Pool(processes=n_processor)
    datasets = rr_partition(data, n_processor)
    for partition in datasets:
        result = pool.apply_async(binary_search, args=(partition,query))
        output = result.get()
        results.append(output)
    return results

parallel_search_date(climateCollection,'2017-12-15',8)

[None,
 None,
 None,
 None,
 {'_id': ObjectId('5b0167e57c376e3639e0039d'),
  'Station': 948702,
  'Date': '2017-12-15',
  'Air Temperature(Celcius)': 18,
  'Relative Humidity': 52.0,
  'WindSpeed  (knots)': 7.1,
  'Max Wind Speed': 14.0,
  'MAX': '74.5*',
  'MIN': '53.1',
  'Precipitation': '0.00I',
  'fires': [ObjectId('693839514c52683454324a4b'),
   ObjectId('354876464441693577726773'),
   ObjectId('76453165416c61484c5a514c'),
   ObjectId('50436f3841437a546b774e52')]},
 None,
 None,
 None]

In [22]:
client.drop_database('fit5148_db')